In [1]:
### INITIALIZATION
### Importing standard modules and date-special modules:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
### Declaring global constants & variables: 
path_politics_2019_03 = 'Data_Files/Source_Files/OD_Politics_2019_03.xlsx' ### Path for 2019-III Data
path_politics_2019_04 = 'Data_Files/Source_Files/OD_Politics_2019_04.xlsx' ### Path for 2019-IV Data
All = slice(None)
list_na_xlsx = ['', '#N/A', '#N/A N/A', '#NA', '-1.#IND', '-1.#QNAN', '-NaN', '-nan', '1.#IND', '1.#QNAN', 'N/A', 'NULL', 'NaN', 'n/a', 'nan', 'null']
list_region_filter = [['DM', 'EM', 'FM'], ['DM'], ['EM'], ['FM']]
list_region_name = ['ISON', 'DM', 'EM', 'FM']
list_back_month = [60, 36, 12, 9, 6, 3, 0]
### Ready datasets importing from MS Excel:
df_politics_2019_03 = pd.read_excel(path_politics_2019_03, index_col = [0, 1], usecols = [0, 1, 2, 7, 12, 14, 17], parse_dates = True, 
                                    na_values = list_na_xlsx, keep_default_na = False).set_index('Market', append = True)
df_politics_2019_04 = pd.read_excel(path_politics_2019_04, index_col = [0, 1], usecols = [0, 1, 2, 7, 9, 11, 14], parse_dates = True, 
                                    na_values = list_na_xlsx, keep_default_na = False).set_index('Market', append = True)
### Data concatenating:
df_politics = pd.concat({'2019_03': df_politics_2019_03, '2019_04': df_politics_2019_04}, join = 'inner', axis = 1).sort_index(level = ['Date', 'Country'])
list_pillar = list(df_politics.columns.get_level_values(1).unique())

In [2]:
### ANALYZING
ser_corr_result = pd.DataFrame(columns = ['Pillar', 'Region', 'Back period', 'X-sect corr mean']).set_index(['Pillar', 'Region', 'Back period']).squeeze()

for iter_pillar in list_pillar:
    ### Pillar looping:
    df_iter_pillar = df_politics.loc[:, (All, iter_pillar)]
#    ### By country correlation:
#    ser_iter_country_corr = df_iter_pillar.groupby('Country').\
#                            apply(lambda iter_group: iter_group.loc[:, ('2019_03', iter_pillar)].corr(iter_group.loc[:, ('2019_04', iter_pillar)]))
#    print(iter_pillar, ': By country correlation:')
#    print('Min:', round(ser_iter_country_corr[ser_iter_country_corr.idxmin() : ser_iter_country_corr.idxmin()], 6))
#    print('Mean:', round(ser_iter_country_corr.mean(), 6))
#    print('\n')
    ### Cross-sectional correlation:
    for iter_region_filter, iter_region_name in zip(list_region_filter, list_region_name):
        ### Region looping:
        df_iter_region = df_iter_pillar.loc[(All, All, iter_region_filter), :].droplevel('Market')
        for iter_period in list_back_month:
            ### Back period looping:
            idx_iter_date_range = pd.date_range(end = df_iter_region.index.get_level_values('Date').max(), periods = iter_period, freq = 'BM')
            date_period_back = df_iter_region.index.get_level_values('Date').max() - pd.offsets.BMonthEnd(iter_period)
#            df_iter_period = df_iter_region.loc[(idx_iter_date_range, All), :]
            df_iter_period = df_iter_region.loc[(date_period_back, All), :]            
            ### By date (cross-sectional) correlation calculating:
            ser_iter_date_corr = df_iter_period.groupby('Date').\
                                    apply(lambda iter_group: iter_group.loc[:, ('2019_03', iter_pillar)].corr(iter_group.loc[:, ('2019_04', iter_pillar)]))
            ser_corr_result.loc[iter_pillar, iter_region_name, iter_period]  = round(ser_iter_date_corr.mean(), 6)
### Results saving:
ser_corr_result.to_excel('Data_Files/Test_Files/Politics_Compare.xlsx', merge_cells = False)

In [83]:
### TEMP

df_test_corr = df_politics.loc[(idx_iter_date_range, All, 'DM'), (All, 'Institutional Robustness (Pillar II)')].droplevel('Market')
df_test_corr.columns = df_test_corr.columns.droplevel(1)
df_test_corr.to_excel('Data_Files/Test_Files/Politics_Compare_Example.xlsx', merge_cells = False)


In [4]:
### TEMP



Timestamp('2019-08-30 00:00:00')